# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook – Pipeline de modélisation et expérimentation multi-modèles


### Identité
- Population cible : Bâtiments non-résidentiels
- Date création : 12 Janvier 2026
- Objectif : Catégoriser les variables pour 3 modèles distincts


### Stratégie 2 Modèles
1. **Modèle 1 (Prédictif Pur)** : Variables autorisées uniquement
2. **Modèle 2 (Data Leakage Partiel)** : + ENERGY STAR Score